# Importazione delle librerie

In [ ]:
import re
import tweepy
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import csv
import string
from tweepy import OAuthHandler
from textblob import TextBlob
from wordcloud import WordCloud
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, words
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from emot.emo_unicode import EMOJI_UNICODE

# Tweets mining

In [ ]:
# keys e tokens dalla Twitter Dev Console
consumer_key = 'HErcO0a3RQxeBc4Au44qZZknQ'
consumer_secret = 'PsC0EhWB5v9DUFVONHIMTpwRJAf211b6Ga77EFdDE9wV2Xah3S'
access_token = '2656923476-BHCZUQTee9C5KufeqFKeQC1rw91YD4O7Xx6CUxi'
access_token_secret = 'uNk9Ue5yCFmn0TxvfrrGxBOO7Z9q4QoAmn346JNS46Sby'

In [ ]:
# prova di autenticazione
try:
	# creazione dell'oggetto OAuthHandler
	auth = OAuthHandler(consumer_key, consumer_secret)
	# impostazione dell'access token e del secret
	auth.set_access_token(access_token, access_token_secret)
	# creazione dell'oggetto API tweepy per il fetch dei tweet
	api = tweepy.API(auth)
except:
	print("Error: Authentication Failed")

## Creazione delle funzione necessaria al mining dei tweet

In questa sezione viene creata la funzione necessaria ad estrarre i tweet da Twitter mediante l'oggetto Cursor di Tweepy. In seguito, i tweet e i parametri a loro associati vengono aggiunti nel file "tweets.csv" per una gestione più modulare.

In [ ]:
def get_tweets(query, count = 10):
	
	# lista vuota per conservare i tweet
	tweets_list = [tweets for tweets in tweepy.Cursor(api.search, q=query, lang="en", tweet_mode='extended').items(count)]

	# scraping dei singoli tweet e scrittura del file tweets.csv
	with open('tweets.csv', 'a', newline='', encoding='utf-8') as csvFile:
		column_names = ['tweet_id', 'text', 'retweet', 'likes']
		csv_writer = csv.DictWriter(csvFile, fieldnames=column_names, delimiter=',')
		csv_writer.writeheader()
		for tweet in tweets_list[::-1]:
			csv_data = {}
			csv_data['tweet_id'] = tweet.id
			csv_data['text'] = tweet.full_text
			csv_data['retweet'] = tweet.retweet_count
			csv_data['likes'] = tweet.favorite_count
			csv_writer.writerow(csv_data)

# utilizzo della funzione per estrarre i tweet secondo la query in questione
get_tweets(query= 'facebook', count = 20)

## Utilizzo del Pandas Dataframe per gestire i tweet estratti all'interno del progetto

In [ ]:
tweets = []

# creazione del DataFrame dal file csv
df = pd.read_csv('tweets.csv', index_col=None, header=0)
tweets.append(df)

tweets_df = pd.concat(tweets, axis=0, ignore_index=True)

tweets_df.head()

## Controlli e ottimizzazione del Dataframe

In [ ]:
tweets_df.shape #controllo delle dimensioni del dataframe

In [ ]:
tweets_df.duplicated(subset='tweet_id').sum() # controllo su valori duplicati (in base all'id)

In [ ]:
tweets_df = tweets_df.drop_duplicates(subset=['tweet_id']) # eliminazione di valori duplicati

In [ ]:
tweets_df.shape # nuovo controllo delle dimensioni

# Processazione dei tweet

Il testo dei tweet contiene molti elementi fuorvianti e inutili in fase di analisi. Qui sotto ci sono alcune funzioni necessarie a ripulirli e ad estrarre solo parti interessanti da analizzare successivamente.

In [ ]:
# definizione delle variabili dai moduli nltk necessari alla processazione
stop_words = list(stopwords.words('english'))
alphabets = list(string.ascii_lowercase)
stop_words = stop_words + alphabets
word_list = words.words()
emojis = list(EMOJI_UNICODE.keys())

In [ ]:
# definizione della funzione necessaria ad eliminare link, caratteri speciali, stopwords,
# emoji, punteggiatura dal testo del tweet e a tokenizzarlo
def process_tweets(tweet):

    tweet = tweet.lower()

    tweet = re.sub(r'http\S+|www\S+|https\S+', '', tweet, flags=re.MULTILINE)

    tweet = re.sub(r'\@\w+|\#|\d+', '', tweet)

    tweet_tokens = word_tokenize(tweet)

    filtered_words = [w for w in tweet_tokens if w not in stop_words]
    filtered_words = [w for w in filtered_words if w not in emojis]
    filtered_words = [w for w in filtered_words if w in word_list]

    no_punc_words = [char for char in filtered_words if char not in string.punctuation]
    no_punc_words = ' '.join(no_punc_words)

    return "".join(no_punc_words)

In [ ]:
# definizione della funzione necessaria ad estrarre gli aggettivi dal testo del tweet
def get_adjectives(tweet):
    tweet = word_tokenize(tweet)
    tweet = [word for (word, tag) in pos_tag(tweet)
        if tag == "JJ"]
    return " ".join(tweet)

In [ ]:
# aggiunta colonna tweet processati al DataFrame
tweets_df['processed tweets'] = tweets_df['text'].apply(process_tweets)

In [ ]:
# aggiunta colonna aggettivi al DataFrame
tweets_df['tweets adjectives'] = tweets_df['processed tweets'].apply(get_adjectives)

In [ ]:
tweets_df.head()

In [ ]:
# definizione della funzione che usa il lemmatizer per riportare le parole dei tweet alla loro forma base
def lemmatize_tweets(tweet):
    tweet_tokens = word_tokenize(tweet)
    lemmatizer = WordNetLemmatizer()
    lemma_words = [lemmatizer.lemmatize(w) for w in tweet_tokens]
    return " ".join(lemma_words)

In [ ]:
# aggiunta colonna tweet 'lemmatizzati' al DataFrame
tweets_df['lemmatized tweets'] = tweets_df['processed tweets'].apply(lemmatize_tweets)

In [ ]:
tweets_df.head()

# Sentiment Analysis

In questa fase viene fatta la sentiment analysis vera e propria tramite TextBlob, che sfrutta il machine learning per determinare la polarità dei tweet forniti su una scala da -1 (negativi) a 1 (positivi), passando per 0 (neutrali).

In [ ]:
# definizione delle funzioni necessarie a calcolare la polarità e il sentiment dei tweet da TextBlob
def get_polarity(tweet):
    return TextBlob(tweet).sentiment.polarity

def get_sentiment(polarity):
    if polarity < 0:
        return "Negative"
    elif polarity == 0:
        return "Neutral"
    else:
        return "Positive"

In [ ]:
# aggiunta colonne polarità e sentiment al DataFrame
tweets_df['polarity'] = tweets_df['lemmatized tweets'].apply(get_polarity)
tweets_df['sentiment'] = tweets_df['polarity'].apply(get_sentiment)

In [ ]:
# DataFrame per il conteggio dei tweet positivi, neutrali e negativi
sentiment = tweets_df['sentiment'].value_counts().rename_axis('sentiment').to_frame('total tweets').reset_index()
sentiment

In [ ]:
# creazione del grafico della distribuzione dei tweet positivi, neutrali e negativi
sentiment_barchart = px.bar(sentiment, x = 'sentiment', y='total tweets', color='sentiment')

sentiment_barchart.update_layout(title='Distribution of Sentiments Results',
                                  margin={"r": 0, "t": 30, "l": 0, "b": 0})

sentiment_barchart.show()

In [ ]:
# creazione stringa con gli aggettivi estratti dai tweet
tweets_string = tweets_df['tweets adjectives'].tolist()
tweets_string = " ".join(tweets_string)

In [ ]:
# creazione della WordCloud contenente gli aggettivi più utilizzati
wc = WordCloud(background_color='white', max_words=1500)

# generazione della WordCloud
wc.generate(tweets_string)

# impostazioni per mostrare la WordCloud
fig = plt.figure()
fig.set_figwidth(20)  # impostazione larghezza
fig.set_figheight(20)  # impostazione altezza

plt.imshow(wc.recolor(random_state=3),
           interpolation="bilinear")
plt.axis('off')
plt.show()

In [ ]:
# creazione lista con gli aggettivi più utilizzati
tweets_string = tweets_df['tweets adjectives'].tolist()
tweets_list=[]
for item in tweets_string:
    item = item.split()
    for i in item:
        tweets_list.append(i)

In [ ]:
# uso del modulo collections per contare gli aggettivi
from collections import Counter
counts = Counter(tweets_list)

# creazione del DataFrame con aggettivo e numero di occorrenze ordinato per numero di occorrenze discendente
df = pd.DataFrame.from_dict(counts, orient='index').reset_index()
df.columns = ['words', 'count']
df.sort_values(by='count', ascending=False, inplace=True)

In [ ]:
df.head(10)